## 문제97. 위의 코드를 수정해서 정확도를 1에폭 돌때 마다 출력되게 하시오!

1 에폭 정확도 0.90  
2 에폭 정확도 0.92  
3 에폭 정확도 0.94  
.  
.  
.  


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

# 1층
x = tf.placeholder("float", [None,784])
W1 = tf.Variable(tf.random_uniform([784,50],-1,1))
b1 = tf.Variable(tf.ones([50]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y) # tensorflow에서는 sigmoid함수를 안만들어도 이렇게 하면 된다.

# 2층 출력층
W2 = tf.Variable(tf.random_uniform([50,10],-1,1))
b2 = tf.Variable(tf.ones([10]))
z = tf.matmul(y_hat, W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder("float", [None,10])
y_label = tf.argmax(y_onehot, axis =1)

correction_prediction = tf.equal(y_predict, y_label) # 같으면 True, 다르면 False
accuracy = tf.reduce_mean(tf.cast(correction_prediction, "float")) # True = 1, False = 0 으로 바꿈

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.001), axis = 1)
result = tf.reduce_mean(loss)
# softmax를 통과한것을 tf.log부분에 넣어야 되기 때문에 z_hat이 들어간다.

# 학습 시키는 코드
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer() # 변수 초기화는 딱 1번만 하면됨.

# 세션 생성
sess = tf.Session()
sess.run(init)

# 데이터 불러오는 부분
for i in range(1,601*20): # 1epoch = 600번
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # batch_xs -> 훈련데이터
    # batch_ys -> 훈련데이터 라벨
    sess.run(train, feed_dict = { x: batch_xs, y_onehot : batch_ys })
    if i % 600 == 0:
        print(int(i/600), '에폭 훈련 정확도', sess.run(accuracy, feed_dict = { x: batch_xs, y_onehot : batch_ys })) # 정확도 출력을 위해 accuracy

## 문제98. underfitting 되고 있는지 확인하기 위해서   
## 테스트 데이터에 대한 정확도도 같이 출력하시오!

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

# 1층
x = tf.placeholder("float", [None,784])
W1 = tf.Variable(tf.random_uniform([784,50],-1,1))
b1 = tf.Variable(tf.ones([50]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y) # tensorflow에서는 sigmoid함수를 안만들어도 이렇게 하면 된다.

# 2층 출력층
W2 = tf.Variable(tf.random_uniform([50,10],-1,1))
b2 = tf.Variable(tf.ones([10]))
z = tf.matmul(y_hat, W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder("float", [None,10])
y_label = tf.argmax(y_onehot, axis =1)

correction_prediction = tf.equal(y_predict, y_label) # 같으면 True, 다르면 False
accuracy = tf.reduce_mean(tf.cast(correction_prediction, "float")) # True = 1, False = 0 으로 바꿈

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.001), axis = 1)
result = tf.reduce_mean(loss)
# softmax를 통과한것을 tf.log부분에 넣어야 되기 때문에 z_hat이 들어간다.

# 학습 시키는 코드
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer() # 변수 초기화는 딱 1번만 하면됨.

# 세션 생성
sess = tf.Session()
sess.run(init)

# 데이터 불러오는 부분
for i in range(1,601*20): # 1epoch = 600번
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # batch_xs -> 훈련데이터
    # batch_ys -> 훈련데이터 라벨
    batch_xt, batch_yt = mnist.test.next_batch(100)
    # batch_xt -> 테스트데이터
    # batch_ys -> 테스트데이터 라벨
    sess.run(train, feed_dict = { x: batch_xs, y_onehot : batch_ys })
    if i % 600 == 0:
        print(int(i/600), '에폭 훈련 정확도', sess.run(accuracy, feed_dict = { x: batch_xs, y_onehot : batch_ys })) # 정확도 출력을 위해 accuracy
        print(int(i/600), '에폭 테스트 정확도', sess.run(accuracy, feed_dict = { x: batch_xt, y_onehot : batch_yt })) # 정확도 출력을 위해 accuracy

## ■ 언더 피팅과 오버 피팅을 막는 방법 구현

1. 언더 피팅을 줄일수 있는 방법

    - 가중치 초기화 설정
        - Xavier (p.206)
        - He (p.206)
    - 배치 정규화

In [ ]:
# 가중치 초기화 방법

# W = tf.Variable(tf.random_uniform([784,10], -1, 1)) # [784,10] 형상을 가진 -1~1 사이의 균등분포 어레이

# W = tf.get_variable(name="W", shape=[784, 10], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값

# W = tf.get_variable(name='W', shape=[784, 10], initializer=tf.contrib.layers.variance_scaling_initializer()) # he 초기값

## 문제99. 점심시간에 생성한 신경망에 가중치 초기화 값을 Xavier로 해서 생성하고 훈련을 시킨 정확도를 확인하시오

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

# 1층
x = tf.placeholder("float", [None,784])
W1 = tf.get_variable(name="W1", shape=[784, 50], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b1 = tf.Variable(tf.ones([50]))

y = tf.matmul(x, W1) + b1
y_hat = tf.nn.relu(y) # tensorflow에서는 sigmoid함수를 안만들어도 이렇게 하면 된다.

# 2층 출력층

W2 = tf.get_variable(name="W2", shape=[50, 10], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b2 = tf.Variable(tf.ones([10]))
z = tf.matmul(y_hat, W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder("float", [None,10])
y_label = tf.argmax(y_onehot, axis =1)

correction_prediction = tf.equal(y_predict, y_label) # 같으면 True, 다르면 False
accuracy = tf.reduce_mean(tf.cast(correction_prediction, "float")) # True = 1, False = 0 으로 바꿈

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.001), axis = 1)
result = tf.reduce_mean(loss)
# softmax를 통과한것을 tf.log부분에 넣어야 되기 때문에 z_hat이 들어간다.

# 학습 시키는 코드
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer() # 변수 초기화는 딱 1번만 하면됨.

# 세션 생성
sess = tf.Session()
sess.run(init)

# 데이터 불러오는 부분
for i in range(1,601*20): # 1epoch = 600번
    batch_xs, batch_ys = mnist.train.next_batch(100)
    # batch_xs -> 훈련데이터
    # batch_ys -> 훈련데이터 라벨
    batch_xt, batch_yt = mnist.test.next_batch(100)
    # batch_xt -> 테스트데이터
    # batch_ys -> 테스트데이터 라벨
    sess.run(train, feed_dict = { x: batch_xs, y_onehot : batch_ys })
    if i % 600 == 0:
        print(int(i/600), '에폭 훈련 정확도', sess.run(accuracy, feed_dict = { x: batch_xs, y_onehot : batch_ys })) # 정확도 출력을 위해 accuracy
        print(int(i/600), '에폭 테스트 정확도', sess.run(accuracy, feed_dict = { x: batch_xt, y_onehot : batch_yt })) # 정확도 출력을 위해 accuracy

## ■ 텐서 플로우로 배치 정규화 구현하는 방법

### 배치정규화란?

__" 신경망 학습시 가중치의 값의 데이터가 골고루 분산될수 있도록 하는것을 강제화 하는 장치 "__ (면접문제)

In [1]:
# 구현코드 : batch_z1 = tf.contrib.layers.batch_norm(z1, True)
#                   ↓
# Affine1 ------> 배치정규화 ------> relu
#  (z1)

## 문제100. 지금까지 완성한 신경망에 배치 정규화를 1층에 구현하시오!

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
tf.reset_default_graph() # 그래프 초기화 코드

# 1층
x = tf.placeholder("float", [None,784])
W1 = tf.get_variable(name="W1", shape=[784, 50], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b1 = tf.Variable(tf.ones([50]))

y = tf.matmul(x, W1) + b1
batch_y = tf.contrib.layers.batch_norm(y, True) # True를 킴으로써 배치정규화를 실시함.
y_hat = tf.nn.relu(batch_y) # tensorflow에서는 sigmoid함수를 안만들어도 이렇게 하면 된다.

# 2층 출력층

W2 = tf.get_variable(name="W2", shape=[50, 10], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b2 = tf.Variable(tf.ones([10]))
z = tf.matmul(y_hat, W2) + b2
z_hat = tf.nn.softmax(z)
y_predict = tf.argmax(z_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder("float", [None,10])
y_label = tf.argmax(y_onehot, axis =1)

correction_prediction = tf.equal(y_predict, y_label) # 같으면 True, 다르면 False
accuracy = tf.reduce_mean(tf.cast(correction_prediction, "float")) # True = 1, False = 0 으로 바꿈

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(z_hat+0.001), axis = 1)
result = tf.reduce_mean(loss)
# softmax를 통과한것을 tf.log부분에 넣어야 되기 때문에 z_hat이 들어간다.

# 학습 시키는 코드
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer() # 변수 초기화는 딱 1번만 하면됨.

# 세션 생성
sess = tf.Session()
sess.run(init)

# 데이터 불러오는 부분
for i in range(1,601*20): # 1epoch = 600번
    batch_xs, batch_ys = mnist.train.next_batch(100) # train
    # batch_xs -> 훈련데이터
    # batch_ys -> 훈련데이터 라벨
    batch_xt, batch_yt = mnist.test.next_batch(100) # test
    # batch_xt -> 테스트데이터
    # batch_ys -> 테스트데이터 라벨
    sess.run(train, feed_dict = { x: batch_xs, y_onehot : batch_ys })
    if i % 600 == 0:
        print(int(i/600), '에폭 훈련 정확도', sess.run(accuracy, feed_dict = { x: batch_xs, y_onehot : batch_ys })) # 정확도 출력을 위해 accuracy
        print(int(i/600), '에폭 테스트 정확도', sess.run(accuracy, feed_dict = { x: batch_xt, y_onehot : batch_yt })) # 정확도 출력을 위해 accuracy

## 문제101. 위의 2층 신경망을 3층신경망으로 변경하고 정확도를 확인하시오!

입력층 -----> 1층 ------> 2층 ------> 3층

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
tf.reset_default_graph() # 그래프 초기화 코드

# 1층
x = tf.placeholder("float", [None,784])
W1 = tf.get_variable(name="W1", shape=[784, 100], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b1 = tf.Variable(tf.ones([100]))
y = tf.matmul(x, W1) + b1
batch_y = tf.contrib.layers.batch_norm(y, True) # True를 킴으로써 배치정규화를 실시함.
y_hat = tf.nn.relu(batch_y) # tensorflow에서는 sigmoid함수를 안만들어도 이렇게 하면 된다.

# 2층

W2 = tf.get_variable(name="W2", shape=[100, 50], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b2 = tf.Variable(tf.ones([50]))
z = tf.matmul(y_hat, W2) + b2
batch_z = tf.contrib.layers.batch_norm(z, True)
z_hat = tf.nn.relu(batch_z)


# 3층 출력층 
W3 = tf.get_variable(name="W3", shape=[50, 10], initializer=tf.contrib.layers.xavier_initializer()) # xavier 초기값
b3 = tf.Variable(tf.ones([10]))
o = tf.matmul(z_hat, W3) + b3
o_hat = tf.nn.softmax(o)
y_predict = tf.argmax(o_hat, axis=1)

# 정확도 확인
y_onehot = tf.placeholder("float", [None,10])
y_label = tf.argmax(y_onehot, axis =1)

correction_prediction = tf.equal(y_predict, y_label) # 같으면 True, 다르면 False
accuracy = tf.reduce_mean(tf.cast(correction_prediction, "float")) # True = 1, False = 0 으로 바꿈

# 오차 확인
loss = -tf.reduce_sum(y_onehot * tf.log(o_hat+0.001), axis = 1)
result = tf.reduce_mean(loss)
# softmax를 통과한것을 tf.log부분에 넣어야 되기 때문에 z_hat이 들어간다.

# 학습 시키는 코드
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer() # 변수 초기화는 딱 1번만 하면됨.

# 세션 생성
sess = tf.Session()
sess.run(init)

# 데이터 불러오는 부분
for i in range(1,601*20): # 1epoch = 600번
    batch_xs, batch_ys = mnist.train.next_batch(100) # train
    # batch_xs -> 훈련데이터
    # batch_ys -> 훈련데이터 라벨
    batch_xt, batch_yt = mnist.test.next_batch(100) # test
    # batch_xt -> 테스트데이터
    # batch_ys -> 테스트데이터 라벨
    sess.run(train, feed_dict = { x: batch_xs, y_onehot : batch_ys })
    if i % 600 == 0:
        print(int(i/600), '에폭 훈련 정확도', sess.run(accuracy, feed_dict = { x: batch_xs, y_onehot : batch_ys })) # 정확도 출력을 위해 accuracy
        print(int(i/600), '에폭 테스트 정확도', sess.run(accuracy, feed_dict = { x: batch_xt, y_onehot : batch_yt })) # 정확도 출력을 위해 accuracy

## ■ CNN 계층 추가하기

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

tf.reset_default_graph()
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


#입력층
x = tf.placeholder("float",[None,784]) # None은 배치를 처리하기 위해서 작성
x1 = tf.reshape(x,[-1,28,28,1]) # 2차원을 4차원으로 변경한다.


#은닉1층(CNN층)
b1 = tf.Variable(tf.ones([32])) # 숫자 1로 채워진 바이어스를 생성
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev = 0.01)) # 
y1 = tf.nn.conv2d(x1, W1, strides=[1,1,1,1], padding = 'SAME') 
# convolution을 tf.nn.conv2d로 나타냄.
# stride가 2라면 = [1,2,2,1]
y1 = y1 + b1
y1 = tf.nn.relu(y1)
y1 = tf.nn.max_pool(y1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME') 
# ksize : kernal(filter) 사이즈

#은닉2층(CNN층)
b2 = tf.Variable(tf.ones([64])) # 숫자 1로 채워진 바이어스를 생성
W2 = tf.Variable(tf.random_normal([5,5,32,64],stddev = 0.01))
y2 = tf.nn.conv2d(y1, W2, strides=[1,1,1,1], padding = 'SAME')
y2 = y2 + b2
y2 = tf.nn.relu(y2)

y2 = tf.nn.max_pool(y2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

#은닉3층(fully connected 층)
b3 = tf.Variable(tf.ones([100]))
W3 = tf.get_variable(name='W3', shape=[7*7*64, 100], initializer=tf.contrib.layers.variance_scaling_initializer())
# W3 는 he로 가중치 초기화를 하는 상황

y3 = tf.reshape(y2, [-1, 7*7*64])
y3 = tf.matmul(y3,W3) + b3
y3 = tf.contrib.layers.batch_norm(y3,True)

#드롭아웃( random 으로 드롭아웃한다.)
keep_prob = tf.placeholder("float")
y3_drop = tf.nn.dropout(y3, keep_prob)

#출력층(4층)
b4 = tf.Variable(tf.ones([10]))
W4 = tf.get_variable(name='W4', shape=[100, 10], initializer=tf.contrib.layers.variance_scaling_initializer()) # he 초기값
y4 = tf.matmul(y3_drop,W4) + b4
y4 = tf.contrib.layers.batch_norm(y4,True)
y_hat = tf.nn.softmax(y4)


#예측값
y_predict = tf.argmax(y_hat,1)


# 라벨을 저장하기 위한 변수 생성
y_onehot = tf.placeholder("float",[None,10])
y_label = tf.argmax(y_onehot, axis = 1)


# 정확도를 출력하기 위한 변수 생성
correct_prediction = tf.equal(y_predict, y_label)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))


# 교차 엔트로피 오차 함수
loss = -tf.reduce_sum(y_onehot * tf.log(y_hat), axis = 1)


# SGD 경사 감소법
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)


# Adam 경사 감소법
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)


# 학습 오퍼레이션 정의
train = optimizer.minimize(loss)


# 변수 초기화
init = tf.global_variables_initializer()
train_acc_list = []
test_acc_list = []


with tf.Session() as sess:
    sess.run(init)
    for j in range(20):
        for i in range(600):
            batch_xs, batch_ys = mnist.train.next_batch(100)
            test_xs, test_ys = mnist.test.next_batch(100)

            sess.run(train, feed_dict={x: batch_xs, y_onehot: batch_ys, keep_prob: 0.8})

            if i == 0:
                train_acc = sess.run(accuracy, feed_dict={x: batch_xs, y_onehot: batch_ys, keep_prob: 1.0})
                test_acc = sess.run(accuracy, feed_dict={x: test_xs, y_onehot: test_ys, keep_prob: 1.0})

                train_acc_list.append(train_acc)
                test_acc_list.append(test_acc)

                print('훈련', str(j + 1) + '에폭 정확도 :', train_acc)
                print('테스트', str(j + 1) + '에폭 정확도 :', test_acc)
                print('-----------------------------------------------')

markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot()
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(min(min(train_acc_list),min(test_acc_list))-0.1, 1.1)
plt.legend(loc='lower right')
plt.show()

## ■ 문제102. 위의 신경망을 아래와 같이 수정하시오

기존층 : conv1 --> pooling --> conv2 --> pooling --> fc1 --> fc2(출력층)  
변경된층 : conv1 --> pooling --> fc1 --> fc2(출력층)  

## ■ cifar10 이란 ?

cifar10은 총 60000개 데이터 셋으로 이루어져 있으며 그 중 50000개는 훈련 데이터이고 10000개가 테스트 데이터이다.  
class는 비행기부터 트럭까지 총 10개로 구성되어있다.   


1. 비행기  
2. 자동차  
3. 새  
4. 고양이  
5. 사슴  
6. 개  
7. 개구리  
8. 말  
9. 배  
10. 트럭  

## ■ cifar10 데이터를 신경망에 로드하는 파이썬 코드 작성

D:\data\cifar10\train100 폴더를 하나 만들고 훈련데이터 100장(1~100번)을 복사해서 붙여넣으시오.

## 문제103. D:\\data\\cifar10\\train100 에서 사진을 불러와서 아래와 같이 이미지 이름을 출력하는 함수를 생성하시오.

train_image = D:\data\cifar10\train100  
print(image_load(train_image))  

['1.png', '10.png', '100.png'......., '99.png']  

In [8]:
import os 
train_image = 'D:\\data\\cifar10\\train100'
    
def image_load(path):
    file_list = os.listdir(path)
    return file_list

print(image_load(train_image))

['1.png', '10.png', '100.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '2.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '3.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '4.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '5.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png', '56.png', '57.png', '58.png', '59.png', '6.png', '60.png', '61.png', '62.png', '63.png', '64.png', '65.png', '66.png', '67.png', '68.png', '69.png', '7.png', '70.png', '71.png', '72.png', '73.png', '74.png', '75.png', '76.png', '77.png', '78.png', '79.png', '8.png', '80.png', '81.png', '82.png', '83.png', '84.png', '85.png', '86.png', '87.png', '88.png', '89.png', '9.png', '90.png', '91.png', '92.png', '93.png', '94.png', '95.png', '96.png', '97.png', '98.png', '99.png']


## 문제104. 위의 결과에서 .png는 빼고 숫자만 출력되게 하시오!

In [10]:
import os 
import re
train_image = 'D:\\data\\cifar10\\train100'
    
def image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for i in file_list:
        a = int(re.sub('[^0-9]', '', i)) #숫자가 아닌것은 ''으로 처리
        file_name.append(a)
    return file_name

print(image_load(train_image))

[1, 10, 100, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 6, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 8, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 9, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


## 문제105. 위의 결과를 정렬되어서 출력되게 하시오!

In [11]:
import os 
import re
train_image = 'D:\\data\\cifar10\\train100'
    
def image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for i in file_list:
        a = int(re.sub('[^0-9]', '', i)) #숫자가 아닌것은 ''으로 처리
        file_name.append(a)
    file_name.sort()
    return file_name

print(image_load(train_image))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


## 문제106. 위의 출력된 결과에 다시 .png를 붙여서 아래와 같이 출력되게 하시오!

In [12]:
import os 
import re
train_image = 'D:\\data\\cifar10\\train100'
    
def image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for i in file_list:
        a = int(re.sub('[^0-9]', '', i)) #숫자가 아닌것은 ''으로 처리
        file_name.append(a)
        file_name.sort()
    file_res = []
    for j in file_name:
        file_res.append('%d.png' %j)
    
    return file_res

print(image_load(train_image))

['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png', '43.png', '44.png', '45.png', '46.png', '47.png', '48.png', '49.png', '50.png', '51.png', '52.png', '53.png', '54.png', '55.png', '56.png', '57.png', '58.png', '59.png', '60.png', '61.png', '62.png', '63.png', '64.png', '65.png', '66.png', '67.png', '68.png', '69.png', '70.png', '71.png', '72.png', '73.png', '74.png', '75.png', '76.png', '77.png', '78.png', '79.png', '80.png', '81.png', '82.png', '83.png', '84.png', '85.png', '86.png', '87.png', '88.png', '89.png', '90.png', '91.png', '92.png', '93.png', '94.png', '95.png', '96.png', '97.png', '98.png', '99.png', '100.png']


## 문제109. 훈련이미지의 라벨을 one hot encoding 하는 코드를 구현하시오!

train_label.csv 파일의 위치를 지정

train_label = 'D:\\data\\cifar10\\train_label.csv'

In [ ]:
import csv
def label_load(path):
    file = open(path)
    labeldata = csv.reader(file)
    labellist = []
    for i in labeldata:
        labellist.append(i)
    return labelist